<a href="https://colab.research.google.com/github/a1koAiko/CSCI446_assignment2/blob/master/AI_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from collections import defaultdict
from random import randint
from json import dumps
import pandas as pd
import numpy as np
from numpy import array, linalg
from time import perf_counter
from collections import deque as q, OrderedDict as od
from queue import PriorityQueue as Q
from math import*

In [0]:
#euclidean distance -> returns true if the locations are 1 unit away from each other
def dist(a,b):
    x=array(a)
    y=array(b)
    return linalg.norm(x-y) < 1.4

In [55]:
#example dist(a,b)
a= (0,0)
b= (1,0)
c= (1,1)

print(f'{a}<->{b} = {dist(a,b)}\n{a}<->{c} = {dist(a,c)}')

(0, 0)<->(1, 0) = True
(0, 0)<->(1, 1) = False


In [0]:
class Maze():
    def __init__(self, file_name ):
        self.name = file_name.replace('.txt', '')
        self.maze = self.load_maze(file_name)
        
    def _maze(self):
        return self.maze
    
    #saves the search results of the maze to a text file
    def save_maze(self, bot):
        maze_csv = self.maze.to_csv(index=False, header=False).replace(',','')
        with open(f'{self.name} - {str(bot)[10:13]} search results.txt', 'w') as f:
            f.writelines(maze_csv) 
    
    #read file and convert text to pandas dataframe
    def load_maze(self, file_name):
        maze = defaultdict(list)
        with open(f'{file_name}') as f:
            lines=f.readlines()
            for row,line in enumerate(lines):
                for col, char in enumerate(line):
                     maze[col].append(char) 
        del maze[list(maze.keys())[-1]]
        return pd.DataFrame(maze, dtype=str)
    
    
    #converts the path to the goal into exclamation marks and saves the file
    def print_solution(self,path,bot):
        for location in path:
            #if != color put color?
            if  self.maze.iloc[location[0]][location[1]] != 'P' and self.maze.iloc[location[0]][location[1]] != '*':
                self.maze.iloc[location[0]][location[1]] = '!'
        maze_csv = self.maze.to_csv(index=False, header=False).replace(',','')
        with open(f'{self.name} - {str(bot)[10:13]} solution.txt', 'w') as f:
            f.writelines(maze_csv) 

In [0]:
class Agent():
    def __init__(self, algorithm,maze):
  
        self.path=q()
        self.pos = None
        self.maze = maze
        self.solution_path = []
        self.search_results = []
                        
    # scans the maze file for 'P'and '*' returns (row, col)
    def get_location_of_(self, item):
        for i in self.maze.maze.columns:
            if len(np.where(self.maze.maze[i]==item)[0]) != 0 :
                j=self.maze.maze.loc[self.maze.maze[i] == item]
                self.pos=(j.index.values[0], i)
                return j.index.values[0], i

    #prints the state of the agent in the maze
    def get_state(self):
        for i in range(-1,2):
            for j in range(-1,2):
                print(f'{self.maze.maze.iloc[self.pos[0]+i][self.pos[1]+j]}' , end='')
            print()
            
    #returns a list of  possible actions for the table        
    def get_actions(self):
        actions = []
        for i in range(-1,2):
            for j in range(-1,2):
                #changed "%" (wall) to "_" (open space)
                if abs(i+j) ==1  and self.maze.maze.iloc[self.pos[0]+i][self.pos[1]+j] != '_':
                    actions.append((self.pos[0]+i, self.pos[1]+j))        
                    
        return actions
    
    #runs the functions neccesary to produce maze searching results
    def search_maze(self):
        print('START: search')
        self.search_results, self.solution_path = self.search_alg(self)
        print('DONE: search')
        print('START: solution')
        self.get_solution()
        print('DONE: solution')
        print('START: save maze')
        self.maze.save_maze(self.search_alg)
        print('DONE: save maze')
        print('START: save solution')
        self.maze.print_solution(self.solution_path, self.search_alg) 
        print('DONE: save solution')
        self.get_results()
   
    #prints out results
    def get_results(self):
        print(f'Search Algorith: {str(self.search_alg)[10:13]} \t Maze: {self.maze.name}')
        print(f'Steps taken: {len(self.search_results)}\t Length of solution: {len(self.solution_path)}')
        print('---')
    
    #determines the solution based on the search algortihm. 
    def get_solution(self):
        print('what solution are you getting?')
        if self.search_alg == dfs:
            print('getting dfs solution')
            dfs_solution(self)
        elif self.search_alg == bfs:
            print('getting bfs solution')
            bfs_solution(self)
        elif self.search_alg == gbfs:
            print('getting gbfs solution')
            gbfs_solution(self)

In [0]:
m = Maze('5x5maze.txt')
agent = Agent(m) #constructor uses an algorithm and a maze

In [149]:
maze_obj = Maze('5x5maze.txt')
print(f'{maze_obj.name}\n{maze_obj.maze}')

5x5maze
  0  1  2  3  4  5  6  7  8  9  ... 18 19 20 21 22 23 24 25 26 27
0  B  _  _  R  O  ,  _  _  _  Y ...  _  R  O  _  G  ,  _  B  G  _

[1 rows x 28 columns]


In [139]:
agent.get_location_of_('B')

(0, 0)

In [132]:
agent.get_location_of_('R')

(0, 3)

In [133]:
agent.get_location_of_('O')

(0, 4)

In [134]:
agent.get_location_of_('Y')

(0, 9)

In [135]:
agent.get_location_of_('G')

(0, 22)

In [151]:
class node():
  file = open('5x5maze.txt', 'r')
  lines = file.readlines()
  
  def cleanlist(lines):
      for y in lines:  # technically don't need because we have create only 1 line now
          array = list(y)
          for val in array:
              if val == (','):
                  array.remove(val)
      lines = array
      maze = []
      tempmaze = []
      for x in range(0, 25):
          tempmaze.append(lines[x])
          if (x == 4 or x == 9 or x == 14 or x == 19 or x == 24):  # needs to be changed
              print(x)
              maze.append(tempmaze)
              tempmaze = []
      return maze
      
      
  Maze = (cleanlist(lines))  # GLOBAL VARIABLE
       
  def printmaze(Maze):
    length = len(Maze)  # GLOBAL VARIABLE
    for x in range(length):
        print('\n')
        for y in range(length):
            print(Maze[x][y], end=" ")
            #print(Maze[x][y].index('B'))
 
  printmaze(Maze)
  
                

4
9
14
19
24


B _ _ R O 

_ _ _ Y _ 

_ _ Y _ _ 

_ R O _ G 

_ B G _ _ 